### Camera Calibration

In [135]:
import numpy as np
import os
import sys
import cv2
import matplotlib.pyplot as plt
import glob
import pickle
%matplotlib inline

camera_calibration_dir = os.path.join(os.getcwd(),"camera_cal")
print("Using Calibration files from Directory:\n ", camera_calibration_dir)

cal_images = glob.glob(os.path.join(camera_calibration_dir, "calibration*.jpg"))
print("Found {} calibration images.".format(len(cal_images)))


Using Calibration files from Directory:
  /home/ankur/UdacityND/CarND-Advanced-Lane-Lines/camera_cal
Found 20 calibration images.


In [115]:
def create_3D_obj_points(nx, ny):
    obj = np.zeros((nx*ny,3), np.float32)
    obj[:, 0:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)
    return obj

In [ ]:
def save_calibration(mtx, dist, pickle_name):#Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
    calibration = {}
    calibration["mtx"] = mtx
    calibration["dist"] = dist
    pickle.dump( calibration, open(pickle_name, "wb")) 

In [116]:
def get_image_corners(image_list, nx = 9, ny = 6, num_images = 20, debug=False):
    image_pts = []
    obj_pts = []
    for fname in cal_images[0:num_images]:
        debug_string = ''
        debug_string += "Image: " + fname + "\n"
        img = cv2.imread(fname)

        # Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #plt.imshow(gray, cmap='gray')
        
        # Find the chessboard corners
        ret, img_corners = cv2.findChessboardCorners(gray, (nx, ny), None)
    
        # If found, draw corners
        if ret:
            # Draw and display the corners
            #cv2.drawChessboardCorners(img, (nx, ny), corners, ret)
            debug_string += ("# of corners found: {}\n".format(len(corners)))
            image_pts.append(img_corners)
            obj_pts.append(create_3D_obj_points(nx, ny))
            #plt.imshow(img)
        else:
            debug_string += ("No corners found\n")
            
        if debug:
            print(debug_string)
    return image_pts, obj_pts
        


In [117]:
#setup calibration details
nx,ny = 9,6
img = cv2.imread(cal_images[0])
img_size = (img.shape[1], img.shape[0])

#Get list of image points and correspondin 3D points
image_points, obj_points = get_image_corners(cal_images, nx, ny)

# Do camera calibration given object points and image points
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points, image_points, img_size,None,None)

#Save calibration file as a pickle
calib_file_name = os.path.join(camera_calibration_dir,"calibration_pickle.p")
save_calibration(mtx, dist, calib_file_name)

In [130]:
# Test undistortion on images...save them to a folder for easy viewing
calibration_output_dir = os.path.join(camera_calibration_dir, "calib_output")

if not os.path.isdir(calibration_output_dir):
    os.mkdir(calibration_output_dir)
    
for fname in cal_images:
    img = cv2.imread(fname)
    img_size = (img.shape[1], img.shape[0])
    name = os.path.basename(fname)
    output_name = os.path.join(calibration_output_dir,name)
    dst_img = cv2.undistort(img, mtx, dist, None, mtx)
    vis_out = np.concatenate((img, dst_img), axis=1)
    ret = cv2.imwrite(output_name,vis_out)

In [142]:
def get_calibration(calib_name):
    calibration = pickle.load(open(calib_name, "rb"))
    print(calibration)
    
calib_name = os.path.join(camera_calibration_dir,"calibration_pickle.p")
get_calibration(calib_name)

{'dist': array([[ -2.41017956e-01,  -5.30721173e-02,  -1.15810355e-03,
         -1.28318856e-04,   2.67125290e-02]]), 'mtx': array([[  1.15396093e+03,   0.00000000e+00,   6.69705357e+02],
       [  0.00000000e+00,   1.14802496e+03,   3.85656234e+02],
       [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00]])}


In [145]:
test_dir = "/home/ankur/UdacityND/CarND-Advanced-Lane-Lines/test_images/"
for fname in glob.glob(os.path.join(test_dir, "*.jpg")):
    img = cv2.imread(fname)
    img_size = (img.shape[1], img.shape[0])
    name = "put" + os.path.basename(fname)
    print(name)
    output_name = os.path.join(test_dir,name)
    dst_img = cv2.undistort(img, mtx, dist, None, mtx)
    vis_out = np.concatenate((img, dst_img), axis=1)
    ret = cv2.imwrite(output_name,vis_out)